In [1]:
# pytorch mlp for binary classification
from numpy import vstack
from pandas import read_csv
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch import Tensor
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import Sigmoid
from torch.nn import Module
from torch.optim import SGD
from torch.nn import BCELoss
from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_

import modeltrack.experiment as exp

# dataset definition
class CSVDataset(Dataset):
    # load the dataset
    def __init__(self, path):
        # load the csv file as a dataframe
        df = read_csv(path, header=None)
        # store the inputs and outputs
        self.X = df.values[:, :-1]
        self.y = df.values[:, -1]
        # ensure input data is floats
        self.X = self.X.astype('float32')
        # label encode target and ensure the values are floats
        self.y = LabelEncoder().fit_transform(self.y)
        self.y = self.y.astype('float32')
        self.y = self.y.reshape((len(self.y), 1))
 
    # number of rows in the dataset
    def __len__(self):
        return len(self.X)
 
    # get a row at an index
    def __getitem__(self, idx):
        return [self.X[idx], self.y[idx]]
 
    # get indexes for train and test rows
    def get_splits(self, n_test=0.33):
        # determine sizes
        test_size = round(n_test * len(self.X))
        train_size = len(self.X) - test_size
        # calculate the split
        return random_split(self, [train_size, test_size])
 

# prepare the dataset
def prepare_data(path, batch):
    # load the dataset
    dataset = CSVDataset(path)
    # calculate split
    train, test = dataset.get_splits()
    # prepare data loaders
    train_dl = DataLoader(train, batch_size=batch, shuffle=True)
    test_dl = DataLoader(test, batch_size=batch, shuffle=False)
    return train_dl, test_dl

# model definition
class MLP(Module):
    # define model elements
    def __init__(self, n_inputs):
        super(MLP, self).__init__()
        # input to first hidden layer
        self.hidden1 = Linear(n_inputs, 10)
        kaiming_uniform_(self.hidden1.weight, nonlinearity='relu')
        self.act1 = ReLU()
        # second hidden layer
        self.hidden2 = Linear(10, 8)
        kaiming_uniform_(self.hidden2.weight, nonlinearity='relu')
        self.act2 = ReLU()
        # third hidden layer and output
        self.hidden3 = Linear(8, 1)
        xavier_uniform_(self.hidden3.weight)
        self.act3 = Sigmoid()
 
    # forward propagate input
    def forward(self, X):
        # input to first hidden layer
        X = self.hidden1(X)
        X = self.act1(X)
         # second hidden layer
        X = self.hidden2(X)
        X = self.act2(X)
        # third hidden layer and output
        X = self.hidden3(X)
        X = self.act3(X)
        return X
    
def train_one_epoch(dataloader, model, loss_fn, optimizer):
    """
    One cycle of model training
    """
    model.train()
    train_loss = 0
    total_correct = 0

    for i, (inputs, targets) in enumerate(dataloader):
        # set previous gradients to zero
        optimizer.zero_grad()

        # Compute prediction for current batch
        preds = model.forward(inputs)
                
        # compute the loss between actual and predicted values
        loss = loss_fn(preds, targets)
        train_loss += loss.item()

        # Backpropagation to compute new gradients
        loss.backward()

        # update the model parameters
        optimizer.step()
        
        #compute the number of correct predictions
        total_correct += (preds.round() == targets).type(torch.float).sum().item()

    # compute the training loss of the epoch
    avg_train_loss = train_loss / len(dataloader)
    train_accuracy = total_correct / len(dataloader.dataset)

    return avg_train_loss, train_accuracy


def test_one_epoch(dataloader, model, loss_fn):
    """
    One cycle of the model testing/validation cycle
    """
    model.eval()
    test_loss = 0
    total_correct = 0

    for i, (inputs, targets) in enumerate(test_dl):
        with torch.no_grad():
            preds = model.forward(inputs)
            loss = loss_fn(preds, targets)
            test_loss += loss.item()
            total_correct += (preds.round() == targets).type(torch.float).sum().item()

    avg_test_loss = test_loss / len(dataloader)
    test_accuracy = total_correct / len(dataloader.dataset)

    return avg_test_loss, test_accuracy

In [ ]:
# instantiate a ModelTracker
config = {
    "description": "This is my first neural net test run",
    "seed": 20,
    "batch_size": 32,
    "learning_rate": 0.01,
    "max_epochs": 10,
    "momentum": 0.9,
    "overwrite": True,
}

tracker = exp.ModelTracker("test-model", config=config)

# prepare the data
path = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/ionosphere.csv'
train_dl, test_dl = prepare_data(path, tracker.config.batch_size)

# define the network
model = MLP(34)
loss_fn = BCELoss()
optimizer = SGD(model.parameters(), lr=tracker.config.learning_rate, momentum=tracker.config.momentum)

# inform tracker that training will commence
tracker.start_training()

# main training loop
for epoch in range(tracker.config.max_epochs):
    # train model
    train_loss, train_acc  = train_one_epoch(train_dl, model, loss_fn, optimizer)

    # evaluate model
    test_loss, test_acc = test_one_epoch(test_dl, model, loss_fn)
    
    tracker.save_epoch_stats(train_loss, test_loss, train_acc, test_acc)
    tracker.save_model(model, epoch, optimizer, test_loss)

tracker.finish_training(model=model)